In [2]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
from c_train_and_test import *
from d_post_training_quantize import *

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb
importing Jupyter notebook from c_train_and_test.ipynb
importing Jupyter notebook from d_post_training_quantize.ipynb


## 1、深入理解卷积中量化的细节

### 1.1 定义一个输入并且输出指定范围数据

In [5]:
a = torch.randn((64,1,28,28))
a[0][0][0][:5]

tensor([0.4792, 0.6436, 1.5960, 0.5424, 0.9110])

### 1.2 定义一个卷积

In [6]:
conv = torch.nn.Conv2d(1, 40, 3, 1)

### 1.3 进行卷积运算并且输出指定范围数据

In [7]:
res = conv(a)
res[0][0][0][:5]

tensor([ 0.7314,  0.8412,  1.1721,  0.4059, -0.0070], grad_fn=<SliceBackward>)

> 上面一步的目的是模拟正常运算，此时网络中的权重和偏置已存在

### 1.4 进行对输入的量化

In [12]:
# 1
min_a, max_a = calcu_max_and_min(a, None, None, False)
# 2
scale_a, zero_point_a = calcu_scale_and_zeropoint(min_a, max_a, 8, False)
# 3
q_a = quantize_tensor(a, scale_a, zero_point_a).int()
# 4
dq_a = dequantize_tensor(q_a, scale_a, zero_point_a)

print("原始：", a[0][0][0][:5])
print("量化：", q_a[0][0][0][:5])
print("反量化：", dq_a[0][0][0][:5])

原始： tensor([0.4792, 0.6436, 1.5960, 0.5424, 0.9110])
量化： tensor([146, 151, 181, 148, 159], dtype=torch.int32)
反量化： tensor([0.4745, 0.6327, 1.5817, 0.5378, 0.8857])


### 1.5 进行对权重的量化
两种量化方式，逐层量化、逐通道量化
#### 1、逐通道量化

In [13]:
# 1、获取权重
w = conv.weight.data
# 2、
min_w, max_w = calcu_max_and_min(w, None, None, True)
# 3、
scale_w, zero_point_w = calcu_scale_and_zeropoint(min_w, max_w, 8, True)
# 4、
q_w = quantize_tensor(w, scale_w, zero_point_w,8,False, True).int()
# 5、
dq_w = dequantize_tensor(q_w, scale_w, zero_point_w, True)

print("原始：", w[0][0][0][:5])
print("量化：", q_w[0][0][0][:5])
print("反量化：", dq_w[0][0][0][:5])

原始： tensor([0.3315, 0.0992, 0.3202])
量化： tensor([254, 156, 249], dtype=torch.int32)
反量化： tensor([0.3297, 0.0989, 0.3179])


#### 2、逐层量化

> 省略对bias的量化

### 1.6 进行对输出的量化

In [14]:
# 1
min_res, max_res = calcu_max_and_min(res, None, None, False)
# 2
scale_res, zero_point_res = calcu_scale_and_zeropoint(min_res, max_res, 8, False)
# 3
q_res = quantize_tensor(res, scale_res, zero_point_res).int()
# 4
dq_res = dequantize_tensor(q_res, scale_res, zero_point_res)

print("原始：", res[0][0][0][:5])
print("量化：", q_res[0][0][0][:5])
print("反量化：", dq_res[0][0][0][:5])

原始： tensor([ 0.7314,  0.8412,  1.1721,  0.4059, -0.0070], grad_fn=<SliceBackward>)
量化： tensor([155, 159, 171, 143, 127], dtype=torch.int32)
反量化： tensor([ 0.7257,  0.8332,  1.1557,  0.4031, -0.0269])


In [11]:
q_w.shape

torch.Size([40, 1, 3, 3])

In [12]:
torch.tensor(zero_point_w)[23]

tensor(117., dtype=torch.float64)

In [13]:
for i in range(len(zero_point_w)):
    conv.weight.data[i] = q_w[i] - zero_point_w[i]

In [14]:
x_2 = q_a - zero_point_a
y_2 = conv(x_2)
y_2.shape

torch.Size([64, 40, 26, 26])

In [15]:
M = [scale_w[i] * scale_a / scale_res for i in range(len(scale_w))]
len(M)

40

In [16]:
for i in range(len(M)):
    y_2[:,i,:,:] = y_2[:,i,:,:] * M[i]
y_2[0][0:5][0][0]

tensor([-16.5199,  36.1009,  -6.6106,  -8.5260,   5.2803, -38.1270, -22.8699,
        -11.2940,   8.4803, -14.0009,   1.4630, -28.0986,   8.9416,  -0.3656,
         34.0861, -11.2549,  26.3759,  -8.0280,  16.1838, -22.1394,  20.9141,
         18.2061, -43.3648,   8.4468,  24.7519,  -2.5970],
       grad_fn=<SelectBackward>)

In [17]:
z_2 = y_2 +zero_point_res
z_2[0][0:5][0][0]

tensor([114.4801, 167.1010, 124.3894, 122.4740, 136.2803,  92.8730, 108.1301,
        119.7060, 139.4803, 116.9991, 132.4630, 102.9014, 139.9416, 130.6344,
        165.0862, 119.7451, 157.3759, 122.9720, 147.1837, 108.8606, 151.9141,
        149.2061,  87.6352, 139.4468, 155.7519, 128.4030],
       grad_fn=<SelectBackward>)

In [18]:
dz_2 = dequantize_tensor(z_2, scale_res, zero_point_res)
dz_2[0][0:5][0][0]

tensor([-0.4203,  0.9186, -0.1682, -0.2169,  0.1344, -0.9701, -0.5819, -0.2874,
         0.2158, -0.3562,  0.0372, -0.7150,  0.2275, -0.0093,  0.8673, -0.2864,
         0.6711, -0.2043,  0.4118, -0.5633,  0.5321,  0.4632, -1.1034,  0.2149,
         0.6298, -0.0661], grad_fn=<SelectBackward>)

In [19]:
a = torch.randn((64,1,28,28))
a.shape

torch.Size([64, 1, 28, 28])

In [20]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
b = conv(a)

In [21]:
qconv = QConv2d(conv, True, True, 8, True)

In [22]:
q_b = qconv(a)

In [23]:
qconv.freeze()

In [24]:
q_x = qconv.q_in.quantize_tensor(a)
q_x = qconv.quantize_inference(q_x)

In [25]:
out = qconv.q_out.dequantize_tensor(q_x)
out[0][0][:5][0]

tensor([-0.5812, -0.7556, -0.3778,  0.0581, -0.1744,  0.6103,  0.2906,  0.4940,
         0.0581, -0.1453, -0.1453,  0.3487,  1.0462,  0.1453, -0.4940, -0.4650,
         0.1453, -0.1453, -0.7265, -0.0291,  0.6394,  0.9590,  0.6684,  0.1453,
         0.1453,  1.0172], grad_fn=<SelectBackward>)

In [26]:
b[0][0][:5][0]

tensor([-0.5767, -0.7625, -0.3758,  0.0535, -0.1850,  0.6198,  0.2780,  0.4998,
         0.0678, -0.1520, -0.1324,  0.3410,  1.0518,  0.1388, -0.4951, -0.4791,
         0.1440, -0.1452, -0.7232, -0.0254,  0.6318,  0.9714,  0.6540,  0.1549,
         0.1344,  1.0298], grad_fn=<SelectBackward>)

In [42]:
a = torch.randn((64,1,28,28)).cuda()
a[0][0][0][0]-1

tensor(-1.4769, device='cuda:0')

In [36]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
conv.cuda()
b = conv(a)

In [39]:
qconv = QConv2d(conv, True, True, 8, True)
qconv.cuda()

QConv2d(
  (conv_module): Conv2d(1, 40, kernel_size=(3, 3), stride=(1, 1))
)

In [41]:
q_b = qconv(a)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
qconv.freeze()

In [ ]:
q_x = qconv.q_in.quantize_tensor(a)
q_x = qconv.quantize_inference(q_x)

In [ ]:
out = qconv.q_out.dequantize_tensor(q_x)
out[0][0][:5][0]

In [ ]:
b[0][0][:5][0]